#Predicao Igpm





In [34]:
pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
!pip install --upgrade pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pandas-2.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.1 which is incompatible.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.30.0 which is incompatible.
python-bcb 0.1.8 requires pandas<2.0.0,>=1.4.4, but you have pandas 2.0.1 which is incompatible.


In [36]:
!pip install python-bcb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.1
    Uninstalling pandas-2.0.1:
      Successfully uninstalled pandas-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.30.0 which is incompatible.


In [ ]:
!pip install pmdarima
##Solucao https://blog.csdn.net/qq_49821869/article/details/128540436
import os
os.kill(os.getpid(),9)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importar bibbliotecas

In [1]:
!pip install sidrapy
import sidrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from pmdarima import arima
import plotly.express as ploty
from datetime import datetime
from datetime import timedelta
import plotly.express as px
import plotly.graph_objs as go
import random

In [3]:
##Biblioteca para permitir que o colab acesse o google drive 
#https://www.youtube.com/watch?v=yl2WiFbZhPs
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/Funcoes_criadas')

In [4]:
## testes de aderencia e estacionaridades
#https://www.statsmodels.org/dev/examples/notebooks/generated/stationarity_detrending_adf_kpss.html
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

In [5]:
%matplotlib inline
sns.set(rc={'figure.figsize':(20,10)})

In [6]:
##Salvar as dependencias deste projeto
#!pip freeze > requirements.txt
##Importando
#!pip install -r requirements.txt

#Importando os dados

In [7]:
!pip install python-bcb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
meta_dados = pd.read_csv('/content/metadados.csv',sep=';')

In [9]:
data_inicio = '2002-12-01'

##Importando dados do Banco central

In [10]:
def consulta_bc(codigo_bcb,inicio):
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  df = pd.read_json(url)
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  df = df[df.index >= inicio]
  return df

In [11]:
data_inicio

'2002-12-01'

In [12]:
##Esse codigo gera um range de datas mensas(MS SIGINIFICA QUE ELE PEGA O PRIMEIRO DIA DO MES) DO INICO DO PERIODO ATE OS DIAS ATUAIS
freq = pd.date_range(start=data_inicio,end=date.today(),freq='MS')

In [13]:
meta_dados

,categoria,variavel,acronimo,unidade,fonte_primaria,fonte_direta,code,transformações necessárias
0,Atividade Econômica,Índice de Atividade Econômica do Banco Central...,ibc_br_sa,Índice,BCB,BCB,24364,NaN
1,Atividade Econômica,Índice de Confiança do Consumidor - Fecomercio,icc,Índice,Fecomercio,BCB,4393,NaN
2,Atividade Econômica,PIB acumulado dos últimos 12 meses - valores c...,pib_12m,R$ Milhões,BCB,BCB,4382,NaN
3,Economia Internacional,CBOE Volatility Index: VIX,cboe_vix,Índice,CBOE,FRED,VIXCLS,NaN
4,Economia Internacional,Crude Oil Prices: West Texas Intermediate (WTI...,oil_price,US$/barrel,EIA,FRED,MCOILWTICO,NaN
...,...,...,...,...,...,...,...,...
106,Taxa de Juros,Selic nominal acumulada no mês anualizada base...,selic,% a.a.,BCB,BCB,4189,NaN
107,Taxa de Juros,Swaps - DI x pré-fixada - 360 dias - média do ...,swaps_di360,% a.a.,B3,IPEADATA,BMF12_SWAPDI36012,NaN
108,Taxa de Juros,Taxa de juros de longo prazo - TJLP,tjlp,% a.a.,BCB,BCB,256,NaN
109,Taxa de Juros,Taxa de juros pré fixada - estrutura a termo -...,juros_ltn,% a.a.,Anbima,IPEADATA,ANBIMA12_TJTLN1212,NaN


In [19]:
dados_banco= pd.DataFrame(index=pd.date_range(start=data_inicio,end=date.today(),freq='MS'))
for i,j in zip(meta_dados[meta_dados['fonte_direta'] == 'BCB']['code'],meta_dados[meta_dados['fonte_direta'] == 'BCB']['code'].index):
  coluna =  meta_dados[meta_dados.index == j]['acronimo']
  dados_banco[list(coluna)] = consulta_bc(i,data_inicio)

ValueError: ignored

In [ ]:
dados_banco['m2'] = dados_banco['m2'].dropna().astype(int)
dados_banco['m3'] = dados_banco['m3'].dropna().astype(int)
dados_banco['m4'] = dados_banco['m4'].dropna().astype(int)
dados_banco['spc_res_prim_12mpib'] = dados_banco['spc_res_prim_12mpib'].dropna().astype(float)

In [ ]:
dados_banco.info()

##Importando dados do IpeaData
https://analisemacro.com.br/economia/indicadores/coletando-dados-do-ipeadata-com-python/#:~:text=Por%20fim%2C%20para%20extrair%20os,traz%20a%20data%20no%20%C3%ADndice.


In [ ]:
!pip install ipeadatapy
import ipeadatapy as ip

In [ ]:
#Lista das series
#ip.list_series()

In [ ]:
##Pesquisando
#ip.list_series('Caged')

In [ ]:
##Mais informacoes sobre o conjunto de dados
#ip.describe('ANBIMA12_TJTLN1212')

In [ ]:
meta_dados[meta_dados['fonte_direta'] == 'IPEADATA']

In [ ]:
"""
lista = ['EIA366_PBRENT366','ANBIMA12_TJTLN1212','SRF12_IR12','ANBIMA12_IBVSP12','ELETRO12_CEET12']
for i,j in zip(meta_dados[meta_dados['fonte'] == 'IPEADATA']['codigo'],meta_dados[meta_dados['fonte'] == 'IPEADATA']['codigo'].index):
  if i == 'EIA366_PBRENT366':
    brent_fob	 = ip.timeseries('EIA366_PBRENT366',yearGreaterThan = int(data_inicio[0:4])-1).iloc[:,-1]
    brent_fob = brent_fob.resample('MS').mean()
    brent_fob = brent_fob[brent_fob.index >= data_inicio]
    dados = pd.concat([dados,brent_fob],axis=1)
    dados.rename(columns={'VALUE (US$)':'brent_fob'},inplace=True) 
  else:
    lista= []
    coluna =  meta_dados[meta_dados.index == j]['acronimo']
    ipeadata = ip.timeseries(i,yearGreaterThan = int(data_inicio[0:4])-1).iloc[:,-1]
    ipeadata = ipeadata[ipeadata.index >= data_inicio]
    dados[list(coluna)[0]] = ipeadata
  """

In [ ]:
##Passei o codigo acima para melhora no CHAT GPD
# Create a list of data sources
"""Observação:

Em vez de usar a zip()função, usei um forloop para percorrer a lista de fontes de dados.
Eu removi a listavariável desnecessária e, em vez disso, usei o tolist()método para converter a codigocoluna em uma lista de fontes de dados.
Para obter a sigla de cada fonte de dados, usei o locmétodo para acessar a linha com o codigovalor correspondente e, em seguida, selecionei a acronimocoluna.
Substituí a list(coluna)[0]variável acronympor para atribuir os dados à coluna correta no dadosDataFrame."""



data_sources = meta_dados[meta_dados['fonte_direta'] == 'IPEADATA']['code'].tolist()

dados_ipea= pd.DataFrame(index=pd.date_range(start=data_inicio,end=date.today(),freq='MS'))
# Iterate through data sources
for i in data_sources:
    acronym = meta_dados.loc[meta_dados['code'] == i, 'acronimo'].values[0]
    ipeadata = ip.timeseries(i, yearGreaterThan=int(data_inicio[0:4])-1).iloc[:,-1]
    ipeadata = ipeadata[ipeadata.index >= data_inicio]
    dados_ipea[acronym] = ipeadata


In [ ]:
dados_ipea.head()

In [ ]:
dados_ipea.info()

##Importando dados de expectativa de inflacao (Focus/BCB)
https://wilsonfreitas.github.io/python-bcb/expectativas.html#bcb.Expectativas

https://analisemacro.com.br/economia/indicadores/como-coletar-dados-do-boletim-focus-com-o-python/

In [ ]:
meta_dados[meta_dados['fonte_direta']=='Focus/BCB']

In [ ]:
!pip install python-bcb

from bcb import Expectativas

ExpectativasMercadoTop5Anuais: Expectativas de mercado anuais para os indicadores do Top 5

ExpectativasMercadoInstituicoes: Expectativas de mercado informadas pelas instituições credenciadas

ExpectativaMercadoMensais: Expectativas de Mercado Mensais

ExpectativasMercadoInflacao12Meses: Expectativas de mercado para inflação nos próximos 12 meses

ExpectativasMercadoTop5Mensais: Expectativas de mercado mensais para os indicadores do Top 5

ExpectativasMercadoTrimestrais: Expectativas de Mercado Trimestrais

ExpectativasMercadoAnuais: Expectativas de Mercado Anuais

In [ ]:
import datetime
def funcao_tratando_dados_focus(inflacao,ipca_expec:pd.DataFrame):
  #Essa funcao faz tratamento nos dados de expectativas de inflacao. Tenta criar colunas com as expectativas de inflacao durante os intervalos de tempo
  #Invertendo as linhas
  dados_ipca = ipca_expec.copy()
  dados_ipca = dados_ipca[::-1]
  #dados_ipca['monthyear'] = pd.to_datetime(dados_ipca['Data']).apply(lambda x: datetime.strftime(x, "%m/%Y"))
  dados_ipca['monthyear'] = pd.to_datetime(dados_ipca['Data']).apply(lambda x: x.strftime("%m/%Y"))
  dados_ipca['Data'] = pd.to_datetime(dados_ipca['Data'])
  ##Criando a coluna com a diferenca entre data de refencia da previsao e o dia reportado no sistema do bc
  dados_ipca['horizonte'] = (pd.to_datetime(dados_ipca['DataReferencia'])-dados_ipca['Data'])/np.timedelta64(int(30.4375), 'D')
  dados_ipca['horizonte'] = dados_ipca['horizonte'].astype(int)

  dados_ipca['DataReferencia'] = pd.to_datetime(dados_ipca['DataReferencia'])
  ##Retirando as datas negativas
  dados_ipca = dados_ipca.query('horizonte > 0 and horizonte < 13')
  #Selecionando as colunas
  dados_ipca = dados_ipca.loc[:,dados_ipca.columns !='Indicador']
  dados_ipca['monthyear'] = pd.to_datetime(dados_ipca['monthyear'],format='%m/%Y')
  #Criando um grupe by por data e horizonte e obtendo a media
  ## pivotando a o grupo por index data e colunas o horizonte
  dados_ipca = dados_ipca.groupby(['monthyear','horizonte']).mean().pivot_table(values='Mediana',index='monthyear',columns='horizonte')
  ## Convertendo o index em date time
  dados_ipca.index = pd.to_datetime(dados_ipca.index)
  ## ordenando os indices de datas 
  dados_ipca = dados_ipca.sort_index()
  #Renomeando as colunas
  dados_ipca.columns = [f"expectativa_{inflacao}_h_{col}" for col in dados_ipca.columns]

  return dados_ipca

In [ ]:
em = Expectativas()

In [ ]:
##Ipca ExpectativaMercadoMensais: Expectativas de Mercado Mensais
# End point
em = Expectativas()
ep = em.get_endpoint('ExpectativaMercadoMensais')

# Dados do IPCA

ipca_expec = (ep.query()
 .filter(ep.Indicador == 'IPCA')
 .filter(ep.Data >= data_inicio)
 .filter(ep.baseCalculo == 0)
 .select(ep.Indicador, ep.Data, ep.Media, ep.Mediana, ep.DataReferencia,ep.baseCalculo)
 .collect()
  )
group_ipca = ipca_expec.groupby(['DataReferencia']).mean()#.strftime('%Y-%m-%d')
group_ipca.index = pd.to_datetime(group_ipca.index)
group_ipca.sort_index(inplace=True)
group_ipca.head()

In [ ]:
# Igpm ExpectativaMercadoMensais: Expectativas de Mercado Mensais
em = Expectativas()
ep = em.get_endpoint('ExpectativaMercadoMensais')

# Dados do igpm

igpm_expec = (ep.query()
 .filter(ep.Indicador == 'IGP-M')
 .filter(ep.Data >= data_inicio)
 .filter(ep.baseCalculo == 0)
 .select(ep.Indicador, ep.Data, ep.Media, ep.Mediana, ep.DataReferencia,ep.baseCalculo)
 .collect()
  )
igpm_horizonte = funcao_tratando_dados_focus('igpm',igpm_expec)
igpm_horizonte.head()

In [ ]:
##Ipca ExpectativasMercadoInflacao12Meses
# End point
em = Expectativas()
ep = em.get_endpoint('ExpectativasMercadoInflacao12Meses')

# Dados do IPCA

ipca_doze_meses = (ep.query()
 .filter(ep.Indicador == 'IPCA')
 .filter(ep.Data >= data_inicio)
 .filter(ep.baseCalculo == 0)
 .select(ep.Indicador, ep.Data, ep.Media, ep.Mediana, ep.baseCalculo)
 .collect()
  )
ipca_doze_meses.index = pd.to_datetime(ipca_doze_meses['Data'])
ipca_doze_meses = ipca_doze_meses.resample('MS').mean()
ipca_doze_meses.head()

In [ ]:
##Igpm ExpectativaMercadoMensais: Expectativas de Mercado Mensais
# End point
em = Expectativas()
ep = em.get_endpoint('ExpectativasMercadoInflacao12Meses')

# Dados do IPCA

igpm_doze_meses = (ep.query()
 .filter(ep.Indicador == 'IGP-M')
 .filter(ep.Data >= data_inicio)
 .filter(ep.baseCalculo == 0)
 .select(ep.Indicador, ep.Data, ep.Media, ep.Mediana, ep.baseCalculo)
 .collect()
  )
#tRANSFOMANDO EM MENSAIS
igpm_doze_meses.index = pd.to_datetime(igpm_doze_meses['Data'])
igpm_doze_meses = igpm_doze_meses.resample('MS').mean()
igpm_doze_meses.head()

## Dados do FRED
fred = '78de3c0ee42b9def43e4fcd0d0ab46b0'

In [ ]:
meta_dados[meta_dados['fonte_direta']=='FRED']

In [ ]:
!pip install fredapi

In [ ]:
from fredapi import Fred
fred = Fred(api_key='78de3c0ee42b9def43e4fcd0d0ab46b0')

In [ ]:
def pegando_fred(nome_serie, nome):
  taxa_cambio_efetiva = fred.get_series(nome_serie)
  taxa_cambio_efetiva = taxa_cambio_efetiva.to_frame()
  taxa_cambio_efetiva.columns = [nome] 
  return taxa_cambio_efetiva

In [ ]:
#Dados CBOE Volatility Index: VIX e trasnformado para mensal
cboe_vix = pegando_fred('VIXCLS','cboe_vix')
cboe_vix.index = pd.to_datetime(cboe_vix.index)
cboe_vix = cboe_vix.resample('MS').mean()
#Dados 	Crude Oil Prices: West Texas Intermediate (WTI) - Cushing, Oklahoma
oil_price = pegando_fred('MCOILWTICO','oil_price')
#Dados Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity
us_tr_10year = pegando_fred('GS10','us_tr_10year')
#Dados Market Yield on U.S. Treasury Securities at 2-Year Constant Maturity
us_tr_2year = pegando_fred('GS2','us_tr_2year')
#Dados Market Yield on U.S. Treasury Securities at 5-Year Constant Maturity, Inflation-Indexed
us_tr_5year_tips = pegando_fred('FII5','us_tr_5year_tips')
#Dados Producer Price Index by Commodity: All Commodities
ppic = pegando_fred('PPIACO','ppic')

##Dados YAHOO

In [ ]:
meta_dados[meta_dados['fonte_direta']=='YAHOO']

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
ticker = yf.Ticker("DX-Y.NYB")  
indice_dolar = ticker.history(period='max')
indice_dolar = indice_dolar.resample('MS').mean()
indice_dolar.index=indice_dolar.index.strftime('%Y-%m-%d')
indice_dolar.index = pd.to_datetime(indice_dolar.index)
indice_dolar.head()

In [ ]:
ticker = yf.Ticker("^BVSP") 
bovespa = ticker.history(period='max')
bovespa = bovespa.resample('MS').mean()
bovespa.index = bovespa.index.strftime('%Y-%m-%d')
bovespa.index = pd.to_datetime(bovespa.index)
bovespa.head()

In [ ]:
ticker = yf.Ticker("EEM") 
emergentes = ticker.history(period='max')
emergentes = emergentes.resample('MS').mean()
emergentes.index = emergentes.index.strftime('%Y-%m-%d')
emergentes.index = pd.to_datetime(emergentes.index)
emergentes.head()

##Dados Secex	
"cgce" = "https://balanca.economia.gov.br/balanca/IPQ/arquivos/Dados_cgce_mensal.csv",
  "valor" = "https://balanca.economia.gov.br/balanca/IPQ/arquivos/Dados_totais_valor_mensal.csv",
  "totais" = "https://balanca.economia.gov.br/balanca/IPQ/arquivos/Dados_totais_mensal.csv"

In [ ]:
meta_dados[meta_dados['fonte_direta'] == 'Secex']

In [ ]:
import pandas as pd
exportacoes = pd.read_csv('/content/Dados_cgce_mensal.csv', encoding='latin1',sep=';')
exportacoes['data']=exportacoes['CO_ANO'].astype(str) + '-' +exportacoes['CO_MES'].astype(str) +'-'+'01'
exportacoes = exportacoes[::-1]
exportacoes = exportacoes[exportacoes['NO_CGCE'].isin(['Bens de Consumo (BC)'])]
exportacoes = exportacoes[exportacoes['TIPO'].isin(['EXP'])]
exportacoes = exportacoes[exportacoes['TIPO_INDICE'].isin(['PRECO'])]
exportacoes['INDICE'] = exportacoes['INDICE'].apply(lambda x:x.replace(',','.')).astype(float)
exportacoes.index = pd.to_datetime(exportacoes.data, format='%Y-%m-%d')

In [ ]:
indice_exportacoes = pd.read_csv('/content/Dados_totais_mensal.csv', encoding='latin1',sep=';')
indice_exportacoes['data']=indice_exportacoes['CO_ANO'].astype(str) + '-' +indice_exportacoes['CO_MES'].astype(str) +'-'+'01'
indice_exportacoes = indice_exportacoes[::-1]
indice_exportacoes['INDICE'] = indice_exportacoes['INDICE'].apply(lambda x:x.replace(',','.')).astype(float)
indice_exportacoes.index = pd.to_datetime(indice_exportacoes.data, format='%Y-%m-%d')
indice_exportacoes

In [ ]:
dessazonalidade = pd.read_csv('/content/Dados_totais_valor_mensal.csv', encoding='latin1',sep=';')
dessazonalidade['data']=dessazonalidade['CO_ANO'].astype(str) + '-' +dessazonalidade['CO_MES'].astype(str) +'-'+'01'
dessazonalidade = dessazonalidade[::-1]
dessazonalidade['VL_FOB'] = dessazonalidade['VL_FOB'].apply(lambda x:x.replace(',','.')).astype(float)
dessazonalidade.index = pd.to_datetime(dessazonalidade.data, format='%Y-%m-%d')
dessazonalidade

In [ ]:
exp = indice_exportacoes[indice_exportacoes['TIPO'].isin(['EXP'])].copy()
exp_preco = exp[exp['TIPO_INDICE'].isin(['PRECO'])]
exp_quantum = exp[exp['TIPO_INDICE'].isin(['QUANTUM'])]

In [ ]:
imp = indice_exportacoes[indice_exportacoes['TIPO'].isin(['IMP'])].copy()
imp_preco = imp[imp['TIPO_INDICE'].isin(['PRECO'])]
imp_quantum = imp[imp['TIPO_INDICE'].isin(['QUANTUM'])]

##Google trends

https://morioh.com/p/ba56e539c7c0

In [ ]:
meta_dados[meta_dados['fonte_direta'] == 'Google Trends']

In [ ]:
!pip install pytrends 

In [ ]:
from pytrends.request import TrendReq
import seaborn
import matplotlib.pyplot as plt

# for styling
seaborn.set_style("darkgrid")

In [ ]:
try:
  from pytrends.request import TrendReq
  import pandas as pd
  import time

  # Definindo os termos de pesquisa
  kw_list = ['seguro desemprego', 'empregos','FGTS','INSS']

  # Definindo as datas de início e fim da pesquisa
  start_date = '2004-01-01'
  end_date = str(date.today())

  # Dividindo a pesquisa em vários períodos de 5 anos
  periods = pd.date_range(start=start_date, end=end_date, freq='5Y')
  if periods[-1] < pd.to_datetime(end_date):
    periods = periods.append(pd.Index([pd.to_datetime(end_date)]))

  # Criando uma instância do TrendReq
  pytrends = TrendReq()

  # Fazendo a pesquisa para cada período de 5 anos
  data = pd.DataFrame()

  for i in range(len(periods)-1):
    timeframe = f"{periods[i].strftime('%Y-%m-%d')} {periods[i+1].strftime('%Y-%m-%d')}"
    pytrends.build_payload(kw_list, timeframe=timeframe)
    temp_data = pytrends.interest_over_time()
    data = pd.concat([data, temp_data])

    # Aguardando alguns segundos para evitar sobrecarga no servidor do Google Trends
    time.sleep(20)

  data = data.resample('MS').sum()
  # Imprimindo os resultados em uma célula
  data
except:
  data = pd.read_csv('/content/dados_google.csv')
  data.index = pd.to_datetime(data['date'])
  data.drop(['Unnamed: 0','date'],axis=1,inplace=True)

In [ ]:
# plot it
plt.plot(data);


In [ ]:
data

##Dado juros ex ante

In [ ]:
meta_dados[meta_dados['fonte_direta'] == 'IPEADATA e BCB']

In [ ]:
!pip install bizdays

In [ ]:
#https://pypi.org/project/pyettj/
!pip install pyettj


In [ ]:
juros_di = ip.timeseries('BMF12_SWAPDI36012')
juros_di = juros_di[juros_di.index >= data_inicio]
juros_di.head()

In [ ]:
juros_ex_ante = juros_di['VALUE ((% a.a.))'] - ipca_doze_meses['Mediana']
juros_ex_ante

##Juntando os dados

In [ ]:
dados = dados_banco.copy()
dados = dados.join(dados_ipea)
dados['ipca_mensal'] = group_ipca['Media']
dados['ipca_doze_mensal'] = ipca_doze_meses['Media']
dados['igpm_doze_mensal'] = igpm_doze_meses['Media']
dados = dados.join(igpm_horizonte)
dados['cboe_vix'] = cboe_vix['cboe_vix']
dados['oil_price'] = oil_price['oil_price']
dados['us_tr_10year'] = us_tr_10year['us_tr_10year']
dados['us_tr_2year'] = us_tr_2year['us_tr_2year']
dados['us_tr_5year_tips'] = us_tr_5year_tips['us_tr_5year_tips']
dados['ppic'] = ppic['ppic']
dados['indice_dolar'] = indice_dolar['Close']
dados['bovespa'] = bovespa['Close']
dados['emergentes'] = emergentes['Close']
dados['exportacoes'] = exportacoes['INDICE']
dados['exp_preco'] = exp_preco['INDICE']
dados['exp_quantum'] = exp_quantum['INDICE']
dados['imp_preco'] = imp_preco['INDICE']
dados['imp_quantum'] = imp_quantum['INDICE']
dados= pd.concat([dados,data['seguro desemprego']],axis=1)
dados= pd.concat([dados,data['empregos']],axis=1)
dados= pd.concat([dados,data['FGTS']],axis=1)
dados= pd.concat([dados,data['INSS']],axis=1)
dados['juros_ex_ante'] = juros_ex_ante

In [ ]:
dados

#Tratando os dados

##Estacionaridade
A TS é dita estacionária se as suas propriedades estatísticas, tais como a média e variância permanecem constantes ao longo do tempo. Mas por que isso é importante? A maioria dos modelos TS trabalha com o pressuposto de que a TS é estacionária. Intuitivamente, podemos entender que se uma TS tem um comportamento particular ao longo de um tempo, há uma probabilidade muito elevada de que seguirá o mesmo comportamento no futuro. Além disso, as teorias relacionadas com séries estacionárias são mais maduras e mais fáceis de implementar, em comparação com séries não-estacionárias.

https://www.vooo.pro/insights/guia-completo-para-criar-time-series-com-codigo-em-python/

In [ ]:
"""x=0
for i in range(len(dados.columns)):
  try:
    fig, ax = plt.subplots(dpi=120)
    ax.plot(dados.iloc[:,i],label=dados.columns[i])
    plt.legend()
    plt.show()
  except:
    try:
      grafico = dados.iloc[:,i].dropna()
      grafico = grafico.astype(int)
      ax.plot(grafico,label=dados.columns[i])
      plt.legend()
      plt.show()
    except:
      grafico = dados.iloc[:,i].dropna()
      grafico = grafico.astype(float)
      ax.plot(grafico,label=dados.columns[i])
      plt.legend()
      plt.show()
  x=x+1
  print(x)"""

In [ ]:
def plotar(dados):
  for i in range(len(dados.columns)):
    fig, ax = plt.subplots(dpi=120)
    ax.plot(dados.iloc[:,i],label=dados.columns[i])
    plt.legend()
    plt.show()

In [ ]:
plotar(dados)

###Teste de estacionaridade Dickey-Fuller
Dickey-Fuller teste: esse é um dos testes estatísticos para verificar Estacionaridade. Aqui, a hipótese nula é que o TS é não-estacionária. Os resultados do teste são compostos por uma estatística de teste e alguns valores críticos para os níveis de confiança da diferença. Se o ‘teste estatístico’ é menor do que o “valor crítico”, podemos rejeitar a hipótese nula e dizer que a série é estacionária. Consulte este artigo para obter detalhes.

In [ ]:
def test_kpss_adf(dados):
  #https://www.statsmodels.org/dev/examples/notebooks/generated/stationarity_detrending_adf_kpss.html
  def kpss_test(timeseries):
    """
    O KPSS é outro teste para verificar a estacionaridade de uma série temporal. As hipóteses nula e alternativa para o teste KPSS são opostas às do teste ADF.

    Hipótese Nula: O processo é estacionário de tendência.

    Hipótese Alternativa: A série tem raiz unitária (a série não é estacionária).

    Uma função é criada para realizar o teste KPSS em uma série temporal.
    """
    #print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
      kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    return kpss_output

  def adf_test(timeseries):

    """
    O teste ADF é usado para determinar a presença de raiz unitária na série e, portanto, ajuda a entender se a série é estacionária ou não. As hipóteses nula e 
    alternativa deste teste são:
    Hipótese Nula: A série tem raiz unitária.
    Hipótese alternativa: A série não tem raiz unitária.
    Se a hipótese nula não for rejeitada, este teste pode fornecer evidências de que a série é não estacionária.
   Uma função é criada para realizar o teste ADF em uma série temporal.
    """
    #print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
      dftest[0:4],
      index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
      dfoutput["Critical Value (%s)" % key] = value
    return dfoutput


  test_est = pd.DataFrame(index=dados.columns,columns = ['Teste_kpss', 'Estacionaria_ou_nâo_kpss','Teste_adf','Estacionaria_ou_nâo_adf'])
  for k in dados.columns:
    tes_kpss = kpss_test(dados[k].dropna())
    test_est.loc[k,test_est.columns[0]] = tes_kpss[1]
    tes_adf = adf_test(dados[k].dropna())
    test_est.loc[k,test_est.columns[2]] = tes_adf[1]
  test_est['Estacionaria_ou_nâo_kpss']=np.where(test_est['Teste_kpss'] >= 0.05,'Estacionaria','Nâo estacionaria')
  test_est['Estacionaria_ou_nâo_adf']=np.where(test_est['Teste_adf'] <= 0.05,'Estacionaria','Nâo estacionaria')
  return test_est
  

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    from statsmodels.tsa.stattools import adfuller
    import pandas as pd
    import matplotlib.pylab as plt
    import seaborn as sns
    import numpy as np
    for i in range(len(timeseries.columns)):
     #Determing rolling statistics
      rolmean = pd.Series(timeseries.iloc[:,i]).rolling(window=12).mean().dropna()
      rolstd = pd.Series(timeseries.iloc[:,i]).rolling(window=12).std().dropna()

      #Plot rolling statistics:
      if type(timeseries[dados.columns[i]][0]) != str:
        orig = plt.plot(timeseries.iloc[:,i], color='blue',label='Original')
        mean = plt.plot(rolmean, color='red', label='Rolling Mean')
        std = plt.plot(rolstd, color='black', label = 'Rolling Std')
        plt.legend(loc='best')
        plt.title(f'Rolling Mean & Standard Deviation na variavel {dados.columns[i]}')
        plt.show(block=False)
    
        #Perform Dickey-Fuller test:
        print (f'Results of Dickey-Fuller Test:Coluna {dados.columns[i]}')
        dftest = adfuller(timeseries.iloc[:,i].dropna(), autolag='AIC')
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        for key,value in dftest[4].items():
          dfoutput['Critical Value (%s)'%key] = value
        print(dfoutput)
      else:
        try:
          timeseries.iloc[:,i] = timeseries.iloc[:,i].astype(int)
          orig = plt.plot(timeseries.iloc[:,i], color='blue',label='Original')
          mean = plt.plot(rolmean, color='red', label='Rolling Mean')
          std = plt.plot(rolstd, color='black', label = 'Rolling Std')
          plt.legend(loc='best')
          plt.title(f'Rolling Mean & Standard Deviation na variavel {dados.columns[i]}')
          plt.show(block=False)
    
          #Perform Dickey-Fuller test:
          print (f'Results of Dickey-Fuller Test:Coluna {dados.columns[i]}')
          dftest = adfuller(timeseries.iloc[:,i].dropna(), autolag='AIC')
          dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
          for key,value in dftest[4].items():
           dfoutput['Critical Value (%s)'%key] = value
          print(dfoutput)
        except:
          timeseries.iloc[:,i] = timeseries.iloc[:,i].astype(float)
          orig = plt.plot(timeseries.iloc[:,i], color='blue',label='Original')
          mean = plt.plot(rolmean, color='red', label='Rolling Mean')
          std = plt.plot(rolstd, color='black', label = 'Rolling Std')
          plt.legend(loc='best')
          plt.title(f'Rolling Mean & Standard Deviation na variavel {dados.columns[i]}')
          plt.show(block=False)
    
          #Perform Dickey-Fuller test:
          print (f'Results of Dickey-Fuller Test:Coluna {dados.columns[i]}')
          dftest = adfuller(timeseries.iloc[:,i].dropna(), autolag='AIC')
          dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
          for key,value in dftest[4].items():
           dfoutput['Critical Value (%s)'%key] = value
          print(dfoutput)
        
    return test_kpss_adf(timeseries)
          

In [ ]:
dados

In [ ]:
test = test_stationarity(dados)

In [ ]:
test

###Verificando a diferenca de estacionaridade
#http://alkaline-ml.com/pmdarima/1.5.1/modules/generated/pmdarima.arima.ndiffs.html

In [ ]:
from pmdarima import arima

In [ ]:
def report_ndiffs (dados,test=['kpss', 'adf', 'pp'],alpha=0.05):
  dat_ndifis = pd.DataFrame(index=dados.columns)
  for i in test:
    dat_n = []
    for j in dados.columns:
      dat_n.append(arima.ndiffs(dados[j].dropna(),alpha,test=i))
    dat_ndifis[i] = dat_n
  result = []
  for k in range(len(dat_ndifis)):
    result.append(np.where(dat_ndifis.iloc[k,0]==dat_ndifis.iloc[k,1],dat_ndifis.iloc[k,0],
         np.where(dat_ndifis.iloc[k,1]==dat_ndifis.iloc[k,2],dat_ndifis.iloc[k,1],
                  np.where(dat_ndifis.iloc[k,0]==dat_ndifis.iloc[k,2],dat_ndifis.iloc[k,2],''))))
  dat_ndifis['Ndifis'] = result
  dat_ndifis.sort_values(by='Ndifis',ascending=False,inplace=True)
  dat_ndifis['Ndifis'] = dat_ndifis['Ndifis'].astype(int)
  return dat_ndifis

In [ ]:
n_difis = report_ndiffs(dados)
n_difis

###Ajustando dados nao estacionarios metodo de diferenciacao

https://www.youtube.com/watch?v=JuG8hwVK5uQ

Modelagem de séries temporais frequentemente exige a aplicação de transformações nas variáveis, tal como a bem conhecida primeira diferença. Formalmente, podemos descrever essa transformação como:


Ou seja, dado uma série temporal regularmente espaçada, subtraímos do valor em t o valor anterior (t-1), obtendo a série dita "na primeira diferença" ou "nas diferenças".

A mudança de nível da série geralmente contorna diversas características "não desejadas" pelo analista como tendência, sazonalidade, etc., mas dificulta a interpretação dos valores - especialmente quando pretende-se comunicá-los para públicos não técnicos. Para este objetivo é de grande utilidade saber como diferenciar uma série temporal e, sobretudo, também como reverter a transformação.

In [ ]:
def corrigindo_nao_estacionaridade(base,n_difis,valor_predicao):
  dados_est = base.copy()
  for i in n_difis[n_difis['Ndifis'] >= 1].index:
    if i == valor_predicao:
      dados_est[i] = dados_est[i]
    else:
      j = 0
      while j < n_difis['Ndifis'][i]:
        dados_est[i] = dados_est[i].diff(periods=1)
        j = j+1
  return dados_est

In [ ]:
dados_est = corrigindo_nao_estacionaridade(dados,n_difis,'igpm')[2:]
dados_est

In [ ]:
report_ndiffs(dados_est)

##Tratando dados faltantes

In [ ]:
dados_sem_faltante = dados_est.fillna(method='ffill')
dados_sem_faltante = dados_est.fillna(dados_sem_faltante.mean())
dados_sem_faltante

In [ ]:
dados_sem_faltante.isnull().sum()[100:]

##Reduzindo os valores atraves do PCA

In [ ]:
dados_sem_igpm = dados_sem_faltante.loc[:,dados_sem_faltante.columns != 'igpm']

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
pca.fit(dados_sem_igpm)
dados_pca = pca.transform(dados_sem_igpm)
dados_pca = pd.DataFrame(dados_pca,index=dados_sem_igpm.index)
dados_pca

In [ ]:
dados_pca['igpm'] = dados_sem_faltante['igpm']
dados_pca

##Dividindo os dados em treino e teste

In [ ]:
treino = dados_pca[dados_pca.index < pd.to_datetime('2021-01-01')]

In [ ]:
treino

In [ ]:
y_treino = treino['igpm'].values
x_treino = treino.loc[:,treino.columns != 'igpm'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#scaler_treino_x = StandardScaler()
#scaler_treino_y = StandardScaler()
# Padronizando os dados
#x_treino = scaler_treino_x.fit_transform(x_treino)
#y_treino = scaler_treino_y.fit_transform(y_treino.reshape(-1,1))

In [ ]:
teste = dados_pca[dados_pca.index >= pd.to_datetime('2021-01-01')]

In [ ]:
y_teste = teste['igpm'].values
x_teste= teste.loc[:,teste.columns != 'igpm'].values

#x_teste = scaler_treino_x.transform(x_teste)
#y_teste = scaler_treino_y.transform(y_teste.reshape(-1,1))

In [ ]:
print('O tamanho do y_treino e ',y_treino.shape)
print('O tamanho do x_treino e ',x_treino.shape)
print('--------------------')
print('O tamanho do y_teste e ',y_teste.shape)
print('O tamanho do x_teste e ',x_teste.shape)
print('--------------------')

#Treinando os modelos
https://goldinlocks.github.io/Time-Series-Cross-Validation/

In [ ]:
from pmdarima.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import KFold, ShuffleSplit, StratifiedKFold, StratifiedShuffleSplit, TimeSeriesSplit

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
y= treino['igpm'].values
X=treino.loc[:,treino.columns != 'igpm'].values

In [ ]:
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10,lim=100):
    """Create a sample plot for indices of a cross-validation object."""
    cmap_data = plt.cm.Paired
    cmap_cv = plt.cm.coolwarm

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["class", "group"]
    ax.set(
        yticks=np.arange(n_splits + 2) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
        xlim=[0, lim],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
rng = np.random.RandomState(0)

In [ ]:
treino.shape

In [ ]:
n_splits = 10
test_size = 12
tscv= TimeSeriesSplit(max_train_size=223,n_splits= 10,test_size=12)
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print("Fold: {}".format(fold))
    print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
    print("\n")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


ng = np.random.RandomState(1338)
#cv, X, y, groups, ax, n_splits
#fig, ax = plt.subplots()
#plot_cv_indices(tscv, X,  y, groups,ax, n_splits,lw=10,lim=len(X))


##Treinando o RandomForestRegressor

In [ ]:
def frame(consjunto_de_dados,nome):
  a = pd.DataFrame(consjunto_de_dados).mean()
  b = pd.DataFrame(a)
  b.columns = [nome]
  return b 

In [ ]:
#https://medium.com/keita-starts-data-science/time-series-split-with-scikit-learn-74f5be38489e
"""Este código executa uma divisão de série temporal usando a função TimeSeriesSplit do módulo model_selection do scikit-learn e aplica um Random Forest Regressor do módulo ensemble do scikit-learn a um conjunto de dados. 
O código também usa a função mean_squared_error do módulo de métricas do scikit-learn para avaliar o desempenho do modelo e os módulos matemático e aleatório.
O código começa inicializando listas vazias para armazenar resultados e criando uma instância da classe TimeSeriesSplit, com max_train_size = 213, n_splits = 10 e test_size = 12.
O código então entra em um loop while, que será executado até 1000 vezes, tentando diferentes combinações de parâmetros para o Random Forest Regressor. Dentro do loop while, o código usa random.choices para selecionar aleatoriamente uma das 
três funções de critério ('squared_error', 'absolute_error', 'friedman_mse') e random.sample para selecionar aleatoriamente um valor max_features, n_estimators, max_depth e min_samples_leaf a partir de uma gama de valores possíveis.
Para cada combinação dos parâmetros, o código executa uma divisão de série temporal usando o objeto TimeSeriesSplit e ajusta o Random Forest Regressor aos dados de treinamento. O código então usa o regressor para prever os dados de validação e 
calcula a raiz do erro quadrático médio (RMSE) entre os dados de validação previstos e reais. O RMSE é anexado a uma lista mean_erro.
Após a divisão da série temporal, o código anexa a combinação de parâmetros usados ​​e a média da lista mean_erro à lista de pontuação.
Por fim, o código cria um dataframe a partir da lista de pontuação, nomeia as colunas do dataframe e classifica o dataframe por mean_square_error em ordem crescente.
"""
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math
import random
tscv = TimeSeriesSplit(max_train_size=223,n_splits= 10,test_size=12)
i = 1
score = []
maxima_cara = X.shape[1]
##X_tr, X_val = X_train[tr_index], X_train[val_index]
#y_tr, y_val = y_train[tr_index], y_train[val_index]
parada = 0
pred_peri_random = []
while parada < 200:
  for cr in random.choices(['squared_error','absolute_error','friedman_mse']):
    for mf in random.sample(list(range(1, maxima_cara)),1):
      for ne in random.sample(list(range(5, 100)),1):
        for md in random.sample(list(range(1, 40)),1):
          for msl in random.sample(list(range(1, 100)),1):
            mean_erro = []
            valores_reais = []
            for train_index, test_index in tscv.split(X):
              X_tr, X_val = X[train_index], X[test_index]
              y_tr, y_val = y[train_index], y[test_index]
              rfr = RandomForestRegressor(random_state=0,
              criterion = cr,
              max_features=int(mf),
              n_estimators=int(ne),
              max_depth=int(md),
              min_samples_leaf=int(msl))
              rfr.fit(X_tr, y_tr)
              mean_erro.append(np.sqrt(mean_squared_error(y_val, rfr.predict(X_val))))
              pred_peri_random.append(rfr.predict(X_val))
              valores_reais.append(y_val)
            score.append([cr,mf, ne, md, msl,np.mean(mean_erro)])
            print(score[-1])
            print(parada)
            parada = parada + 1
result_random = pd.DataFrame.from_dict(score)
result_random.columns = ['criterion','max_features', 'n_estimators','max_depth','min_samples_leaf','mean_square_error']
result_random.sort_values(by='mean_square_error',inplace=True)
predicao_por_periodo = frame(pred_peri_random,'random')
predicao_por_periodo['igpm'] = frame(valores_reais,'igpm')

In [ ]:
result_random

In [ ]:
predicao_por_periodo

In [ ]:
result_random.to_csv('result_random.csv')

##Treinando o Reggressao lasso

In [ ]:
#https://medium.com/keita-starts-data-science/time-series-split-with-scikit-learn-74f5be38489e
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import math
from sklearn import linear_model
tscv = TimeSeriesSplit(max_train_size=213,n_splits= 10,test_size=12)
i = 1
score = []
pred_peri_lasso = []
valores_re = []
for alpha in np.linspace(0.01, 2, 500):
  for fit_inter in random.choices(['False', 'True']):
    mean_erro = []
    for train_index, test_index in tscv.split(X):
      X_tr, X_val = X[train_index], X[test_index]
      y_tr, y_val = y[train_index], y[test_index]
      lasso = linear_model.Lasso(random_state=0,alpha=alpha)
      lasso.fit(X_tr, y_tr)
      mean_erro.append(np.sqrt(mean_squared_error(y_val, lasso.predict(X_val))))
      pred_peri_lasso.append(lasso.predict(X_val))   
    score.append([alpha, 
                  fit_inter,
                  np.mean(mean_erro)])
    print(score[-1])
result_lasso = pd.DataFrame.from_dict(score)
result_lasso.columns = ['alpha', 'fit_inter','mean_square_error']
result_lasso.sort_values(by='mean_square_error',inplace=True)
predicao_por_periodo['lasso'] = frame(pred_peri_lasso,'lasso')

In [ ]:
fit_inter

In [ ]:
result_lasso

In [ ]:
predicao_por_periodo

In [ ]:
result_lasso.to_csv('result_lasso.csv')

## Treinando gradiente Bosting 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import math
import random
tscv = TimeSeriesSplit(max_train_size=213,n_splits= 10,test_size=12)
i = 1
score = []
pred_peri_gd = []
maxima_cara = X.shape[1]
parada = 0
while parada < 100:
  for los in random.choices(['squared_error', 'absolute_error', 'huber', 'quantile']):
    for rate in np.random.uniform(0.01,2,1):
      for ne in random.sample(list(range(10, 500)),1):
        for md in random.sample(list(range(1, 40)),1):
          for msl in random.sample(list(range(1, 100)),1):
            for crit in random.choices(['friedman_mse', 'squared_error']):
              mean_erro = []
              for train_index, test_index in tscv.split(X):
                X_tr, X_val = X[train_index], X[test_index]
                y_tr, y_val = y[train_index], y[test_index]
                grad_bost = GradientBoostingRegressor(random_state=0,
                loss = los,
                learning_rate = rate,
                criterion=crit,
                max_features=int(mf),
                n_estimators=int(ne),
                max_depth=int(md),
                min_samples_leaf=int(msl))
                grad_bost.fit(X_tr, y_tr)
                mean_erro.append(np.sqrt(mean_squared_error(y_val, grad_bost.predict(X_val))))
                pred_peri_gd.append(grad_bost.predict(X_val))
              score.append([los,rate,mf, ne, md, msl,crit,np.mean(mean_erro)])
              print(score[-1])
              print(parada)
              parada = parada + 1
result_gradi_bostin = pd.DataFrame.from_dict(score)
result_gradi_bostin.columns = ['loss','learning_rate','max_features', 'n_estimators','max_depth','min_samples_leaf','criterio','mean_square_error']
result_gradi_bostin.sort_values(by='mean_square_error',inplace=True)
predicao_por_periodo['gradiente_bosting'] = frame(pred_peri_gd,'gradiente_bosting')

In [ ]:
result_gradi_bostin

In [ ]:
result_gradi_bostin.to_csv('result_gradi_bostin.csv')

#Comparando os modelos

##Comaprando as predicoes por periodos

In [ ]:
igpm_horizonte.mean()

In [ ]:
predicao_por_periodo['focus']= frame(igpm_horizonte,'focus').values

In [ ]:
predicao_por_periodo.index = ['1_periodo','2_periodo','3_periodo','4_periodo','5_periodo',
                                '6_periodo','7_periodo','8_periodo','9_periodo','10_periodo','11_periodo','12_periodo']

In [ ]:
predicao_por_periodo

In [ ]:
plt.plot(predicao_por_periodo.index,predicao_por_periodo.igpm,label='igpm',linewidth=3)
plt.plot(predicao_por_periodo.index,predicao_por_periodo.random,label='random',linewidth=3)
plt.plot(predicao_por_periodo.index,predicao_por_periodo.lasso,label='lasso',linewidth=3)
plt.plot(predicao_por_periodo.index,predicao_por_periodo.gradiente_bosting,label='gradiente_bosting',linewidth=3)
plt.plot(predicao_por_periodo.index,predicao_por_periodo.focus,label='focus',linewidth=3)
plt.title('Prediçâo por horizonte preditivo', fontdict={'size': 25})
plt.xlabel('Periodos de tempo',fontdict={'size': 25})
plt.ylabel('Inflaçâo Mensal',fontdict={'size': 25})
plt.legend();

In [ ]:
rmse_periodos = pd.DataFrame()
for i in predicao_por_periodo.columns:
  if i == 'ipca':
    None
  else:
    rmse_periodos[i] = np.sqrt((predicao_por_periodo['igpm'] - predicao_por_periodo[i])**2)

In [ ]:
rmse_periodos

In [ ]:
plt.plot(rmse_periodos.index,rmse_periodos.focus,label='focus',linewidth=3)
plt.plot(rmse_periodos.index,rmse_periodos.random,label='random',linewidth=3)
plt.plot(rmse_periodos.index,rmse_periodos.lasso,label='lasso',linewidth=3)
plt.plot(rmse_periodos.index,rmse_periodos.gradiente_bosting,label='gradiente_bosting',linewidth=3)
plt.title('RMSE por horizonte preditivo', fontdict={'size': 25})
plt.xlabel('Periodos de tempo',fontdict={'size': 25})
plt.ylabel('RMSE',fontdict={'size': 25})
plt.legend();

##Avaliando os residuos dos medelos

https://medium.com/@krzysztofdrelczuk/acf-autocorrelation-function-simple-explanation-with-python-example-492484c32711

###Regressao lasso

In [ ]:
##Pegando os melhores paramentros
for j in result_lasso.columns:
  if j == 'alpha':
    alpha = result_lasso.loc[result_lasso.index[0],j]
  if j == 'fit_inter':
    fit_inter = result_lasso.loc[result_lasso.index[0],j]
  else:
    None
print(f'Alpha:{alpha}')
print(f'fit_inter:{fit_inter}')

In [ ]:
lasso = linear_model.Lasso(random_state=0,alpha=alpha)
lasso.fit(x_treino, y_treino)

In [ ]:
predic_treino = lasso.predict(x_treino)

In [ ]:
residuo_lasso = y_treino - predic_treino

In [ ]:
plt.hist(residuo_lasso);

In [ ]:
import statsmodels.api as sm

Gráfico ACF de resíduos
Com dados de séries temporais, é altamente provável que o valor de uma variável observada no período de tempo atual seja semelhante ao seu valor no período anterior, ou mesmo no período anterior a esse, e assim por diante. Portanto, ao ajustar um modelo de regressão a dados de séries temporais, é comum encontrar autocorrelação nos resíduos. Nesse caso, o modelo estimado viola a suposição de não haver autocorrelação nos erros, e nossas previsões podem ser ineficientes — sobra alguma informação que deve ser contabilizada no modelo para obter melhores previsões. As previsões de um modelo com erros autocorrelacionados ainda não são viesadas e, portanto, não estão “erradas”, mas geralmente terão intervalos de previsão maiores do que o necessário. Portanto, devemos sempre olhar para um gráfico ACF dos resíduos.

In [ ]:
sm.graphics.tsa.plot_acf(residuo_lasso,lags=40)
plt.show()

###Random forest

In [ ]:
##Pegando os melhores paramentros
for j in result_random.columns:
  if j == 'criterion':
    criterion = result_random.loc[result_random.index[0],j]
  if j == 'max_features':
    max_features = result_random.loc[result_random.index[0],j]
  if j == 'n_estimators':
    n_estimators	 = result_random.loc[result_random.index[0],j]
  if j == 'max_depth':
    max_depth = result_random.loc[result_random.index[0],j]
  if j == 'min_samples_leaf':
    min_samples_leaf = result_random.loc[result_random.index[0],j]
  else:
    None
print(f'criterion:{criterion}')
print(f'max_features:{max_features}')
print(f'n_estimators:{n_estimators}')
print(f'max_depth:{max_depth}')
print(f'min_samples_leaf:{min_samples_leaf}')

In [ ]:
result_random.head()

In [ ]:
rfr = RandomForestRegressor(random_state=0,
              criterion = criterion,
              max_features=max_features,
              n_estimators=n_estimators,
              max_depth=max_depth,
              min_samples_leaf=min_samples_leaf)

In [ ]:
rfr.fit(x_treino, y_treino)

In [ ]:
predic_rando_treino = rfr.predict(x_treino)

In [ ]:
residuo_random = y_treino-predic_rando_treino

In [ ]:
plt.hist(residuo_random);

In [ ]:
sm.graphics.tsa.plot_acf(residuo_random,lags=40)
plt.show()

###Gradiente Boosting

In [ ]:
result_gradi_bostin.head()

In [ ]:
##Pegando os melhores paramentros
for j in result_gradi_bostin.columns:
  if j == 'loss':
    loss = result_gradi_bostin.loc[result_gradi_bostin.index[0],j]
  if j == 'max_features':
    max_features = result_gradi_bostin.loc[result_gradi_bostin.index[0],j]
  if j == 'n_estimators':
    n_estimators	 = result_gradi_bostin.loc[result_gradi_bostin.index[0],j]
  if j == 'max_depth':
    max_depth = result_gradi_bostin.loc[result_gradi_bostin.index[0],j]
  if j == 'min_samples_leaf':
    min_samples_leaf = result_gradi_bostin.loc[result_gradi_bostin.index[0],j]
  if j == 'learning_rate':
    learning_rate = result_gradi_bostin.loc[result_gradi_bostin.index[0],j]
  if j == 'criterio':
    criterio = result_gradi_bostin.loc[result_gradi_bostin.index[0],j]
  else:
    None
print(f'criterion:{criterion}')
print(f'max_features:{max_features}')
print(f'n_estimators:{n_estimators}')
print(f'max_depth:{max_depth}')
print(f'min_samples_leaf:{min_samples_leaf}')
print(f'learning_rate:{learning_rate}')
print(f'loss:{loss}')

In [ ]:
grad_bost = GradientBoostingRegressor(random_state=0,
                loss = loss,
                learning_rate = learning_rate,
                criterion=criterion,
                max_features=max_features,
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_leaf=min_samples_leaf)
grad_bost.fit(x_treino, y_treino)

In [ ]:
predict_grad_treino = grad_bost.predict(x_treino)

In [ ]:
residuo_grad = y_treino - predict_grad_treino

In [ ]:
plt.hist(residuo_grad,bins=50);

In [ ]:
sm.graphics.tsa.plot_acf(residuo_grad,lags=40)
plt.show()


##Avaliando os dados de treino

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_regression(y_true, y_pred, algorithm):
    metrics = {}
    metrics['MAE'] = mean_absolute_error(y_true, y_pred)
    metrics['MSE'] = mean_squared_error(y_true, y_pred)
    metrics['RMSE'] = np.sqrt(metrics['MSE'])
    metrics['R²'] = r2_score(y_true, y_pred)
    metrics_df = pd.DataFrame(metrics, index=[algorithm])
    return metrics_df

In [ ]:
metrics_random_treino = evaluate_regression(y,predic_rando_treino, 'Random Forest Regression')
metrics_linear_treino = evaluate_regression(y,predic_treino, 'Linear Regression')
metrics_grad_treino = evaluate_regression(y,predict_grad_treino, 'Gradiente Boosting Regression')
metrics_treino = pd.concat([metrics_random_treino, metrics_linear_treino,metrics_grad_treino])
metrics_treino

In [ ]:
#Plotando as predicoes
plt.plot(treino.index,y,label='Dados Reais')
plt.plot(treino.index,predic_treino,label='Regressão Lasso')
plt.plot(treino.index,predic_rando_treino,label='Predição Random Forest')
plt.plot(treino.index,predict_grad_treino,label='Gradient Boosting')
plt.legend();

##Previsao com dados de validacao

In [ ]:
predict_lasso_vali = lasso.predict(x_teste)
predict_random_vali = rfr.predict(x_teste)
predict_gradiente_vali = grad_bost.predict(x_teste)

In [ ]:
metrics_random_vali = evaluate_regression(y_teste,predict_random_vali, 'Random Forest Regression')
metrics_grad_vali = evaluate_regression(y_teste,predict_gradiente_vali, 'Gradiente Boosting Regression')
metrics_linear_vali = evaluate_regression(y_teste,predict_lasso_vali, 'Reggessao lasso')
metrics_treino = pd.concat([metrics_random_vali, metrics_linear_vali,metrics_grad_vali])
metrics_treino

In [ ]:
teste

In [ ]:
plt.plot(teste.index,y_teste, label='IGPM',linewidth=3)
plt.plot(teste.index,predict_lasso_vali, label='LASSO',linewidth=3)
plt.plot(teste.index,predict_random_vali, label='RANDOM FOREST',linewidth=3)
plt.plot(teste.index,predict_gradiente_vali, label='GRADIENTE BOOSTING',linewidth=3)
plt.legend();

#Treinando os modelos com grid search

#Treinando os modelos

##Aplicando Grid Search


###Treinando o RandomForestRegressor

RandomForestRegressor(criterion='absolute_error', max_depth=1, max_features=23,
                      n_estimators=20)

In [ ]:
from scipy.stats import randint as sp_randint

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
# definindo os hiperparâmetros
param_grid = {
  "n_estimators": [10, 20,50,60,100],
  "max_depth": [1,2,30,70,100],
  "min_samples_split": [2, 6,8,11],
  "min_samples_leaf": [1,4,9,11],
  "max_features": [1, 3,10,15,23],
  'criterion':['squared_error','absolute_error','friedman_mse']
}

# definição do RMSE como métrica de avaliação



# instanciando o modelo de random forest
rf = RandomForestRegressor()

# instanciando o randomized search
#random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, scoring= ['neg_root_mean_squared_error','neg_mean_absolute_percentage_error','neg_mean_absolute_error'], refit=False, cv=5, verbose=3)

search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring= 'neg_root_mean_squared_error', cv=5, verbose=3)


# fit no modelo
search.fit(x_treino, y_treino)

# print dos melhores hiperparâmetros



In [ ]:
search.best_estimator_

###Treinando o Reggressao lasso

Model: Lasso
'alpha': 0.001

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

models = [LinearRegression(), Ridge(), Lasso()]
param_grid = [
    {'fit_intercept': [True, False]},
    {'alpha': np.logspace(-3, 3, 1000)},
    {'alpha': np.logspace(-3, 3, 1020)}]
for model, param_grid_model in zip(models, param_grid):
    grid_search = GridSearchCV(model, param_grid_model, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
    grid_search.fit(x_treino, y_treino)
    print(f"Model: {type(model).__name__}")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_}")
    print("")


### Treinando gradiente Bosting 

{'learning_rate': 0.01,
 'loss': 'huber',
 'max_depth': 3,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 100}

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
# instanciar o modelo de Gradient Boosting
model = GradientBoostingRegressor()

# definir os hiperparâmetros a serem testados
param_grid = {
    "loss":["ls", "lad", "huber", "quantile"],
    "learning_rate":[0.1, 0.05, 0.01],
    "n_estimators":[100, 250, 500],
    "max_depth":[3, 5, 7, 9],
    "min_samples_split":[2, 4, 8],
    "min_samples_leaf":[1, 2, 4]
}

# instanciar o GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error',verbose=3)

# fit no modelo
grid_search.fit(x_treino, y_treino)

# print dos melhores hiperparâmetros
print("Melhores hiperparâmetros: ", grid_search.best_params_)


##Treinando os modelos com os melhores paramentros

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_regression(y_true, y_pred, algorithm):
    metrics = {}
    metrics['MAE'] = mean_absolute_error(y_true, y_pred)
    metrics['MSE'] = mean_squared_error(y_true, y_pred)
    metrics['RMSE'] = np.sqrt(metrics['MSE'])
    metrics['R²'] = r2_score(y_true, y_pred)
    metrics_df = pd.DataFrame(metrics, index=[algorithm])
    return metrics_df


In [ ]:
#'alpha': 0.6179759029126652
from sklearn.linear_model import LinearRegression, Ridge, Lasso
lasso = Lasso(random_state=0,alpha=0.001)
lasso.fit(x_treino, y_treino)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(criterion='absolute_error', max_depth=1, max_features=23, n_estimators=20)
rfr.fit(x_treino, y_treino)

In [ ]:
melhores_parametros = {'learning_rate': 0.01, 'loss': 'huber', 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 100}

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
grad_bost = GradientBoostingRegressor(random_state=0,learning_rate= 0.01, loss ='huber', max_depth= 3,min_samples_leaf=2, min_samples_split = 8, n_estimators = 100)
grad_bost.fit(x_treino, y_treino)

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=40,activation='relu',input_shape=(x_treino.shape[1],),activity_regularizer='l1'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=40,activation='relu',activity_regularizer='l2'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=40,activation='relu',activity_regularizer='l2'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=40,activation='relu',activity_regularizer='l2'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=30,activation='relu',activity_regularizer='l2'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=25,activation='relu',activity_regularizer='l2'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=20,activation='relu',activity_regularizer='l2'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=10,activation='relu',activity_regularizer='l2'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=1,activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(x_treino, y_treino, epochs=1000,batch_size=20)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=40, input_shape=(x_treino.shape[1],)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=40))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=40))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=40))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=30))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=25))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=20))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=10))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(units=1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(x_treino, y_treino, epochs=1000, batch_size=20)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Cria o modelo
class Modelo(nn.Module):
    def __init__(self):
        super(Modelo, self).__init__()
        self.camada1 = nn.Linear(x_treino.shape[1], 100)
        self.bn1 = nn.BatchNorm1d(100)
        self.camada2 = nn.Linear(100, 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.camada3 = nn.Linear(100, 100)
        self.bn3 = nn.BatchNorm1d(100)
        self.camada4 = nn.Linear(100, 50)
        self.bn4 = nn.BatchNorm1d(50)
        self.camada5 = nn.Linear(50, 40)
        self.bn5 = nn.BatchNorm1d(40)
        self.camada6 = nn.Linear(40, 30)
        self.bn6 = nn.BatchNorm1d(30)
        self.camada7 = nn.Linear(30, 1)
        self.dropout = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.3)
        self.dropout3 = nn.Dropout(p=0.3)
        self.dropout4 = nn.Dropout(p=0.3)
        self.dropout5 = nn.Dropout(p=0.3)
  

    def forward(self, x):
        x = torch.relu(self.bn1(self.camada1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.camada2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.camada3(x)))
        x = self.dropout3(x)
        x=  torch.relu(self.bn4(self.camada4(x)))
        x = self.dropout4(x)
        x=  torch.relu(self.bn5(self.camada5(x)))
        x = self.dropout5(x)
        x=  torch.relu(self.bn6(self.camada6(x)))
        x = self.camada7(x)
        return x

# Define a função de perda e o otimizador
criterio = nn.MSELoss()
modelo = Modelo()
otimizador = torch.optim.Adam(modelo.parameters(),lr=0.1)
#otimizador = torch.optim.SGD(modelo.parameters(), lr=0.01, momentum=0.3)

# Converte os dados em tensores do PyTorch
x_treino_tensor = torch.Tensor(x_treino)
y_treino_tensor = torch.Tensor(y_treino)


# Cria um DataLoader para o conjunto de treino
train_dataset = TensorDataset(x_treino_tensor, y_treino_tensor)
train_loader = DataLoader(train_dataset, batch_size=15, shuffle=True)

# Treina o modelo
for epoch in range(5000):
    # Passa os dados pelo modelo
    for x_batch, y_batch in train_loader:
        saida = modelo(x_batch)
        # Zera os gradientes
        otimizador.zero_grad()
        # Calcula a perda
        perda = criterio(saida, y_batch)
        # Calcula os gradientes e faz o backpropagation
        perda.backward()
        # Atualiza os parâmetros do modelo
        otimizador.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {perda.item():.4f}')


In [ ]:
x_batch.shape,y_batch

##Avaliando os residuos dos medelos

https://medium.com/@krzysztofdrelczuk/acf-autocorrelation-function-simple-explanation-with-python-example-492484c32711

###Regressao lasso


In [ ]:
predic_treino_lasso = lasso.predict(x_treino)

In [ ]:
residuo_lasso = y_treino - predic_treino_lasso

In [ ]:
plt.hist(residuo_lasso);

In [ ]:
import statsmodels.api as sm

Gráfico ACF de resíduos
Com dados de séries temporais, é altamente provável que o valor de uma variável observada no período de tempo atual seja semelhante ao seu valor no período anterior, ou mesmo no período anterior a esse, e assim por diante. Portanto, ao ajustar um modelo de regressão a dados de séries temporais, é comum encontrar autocorrelação nos resíduos. Nesse caso, o modelo estimado viola a suposição de não haver autocorrelação nos erros, e nossas previsões podem ser ineficientes — sobra alguma informação que deve ser contabilizada no modelo para obter melhores previsões. As previsões de um modelo com erros autocorrelacionados ainda não são viesadas e, portanto, não estão “erradas”, mas geralmente terão intervalos de previsão maiores do que o necessário. Portanto, devemos sempre olhar para um gráfico ACF dos resíduos.

In [ ]:
sm.graphics.tsa.plot_acf(residuo_lasso,lags=40)
plt.show()

###Random forest
RandomForestRegressor(max_depth=100, max_features=15, min_samples_leaf=4,
                      min_samples_split=8, n_estimators=20)

In [ ]:
predic_rando_treino = rfr.predict(x_treino)

In [ ]:
residuo_random = y_treino-predic_rando_treino

In [ ]:
plt.hist(residuo_random);

In [ ]:
sm.graphics.tsa.plot_acf(residuo_random,lags=40)
plt.show()

###Gradiente Boosting


In [ ]:
predict_grad_treino = grad_bost.predict(x_treino)

In [ ]:
residuo_grad = y_treino - predict_grad_treino

In [ ]:
plt.hist(residuo_grad,bins=50);

In [ ]:
sm.graphics.tsa.plot_acf(residuo_grad,lags=40)
plt.show()

###Redes neurais Tensor Flow

In [ ]:
predict_redes_treino = model.predict(x_treino)

In [ ]:
#Retirando um dimensao do array
predict_redes_treino = np.squeeze(predict_redes_treino)
predict_redes_treino

In [ ]:
residuo_neurais = y_treino - predict_redes_treino

In [ ]:
plt.hist(residuo_neurais,bins=50);

In [ ]:
sm.graphics.tsa.plot_acf(residuo_neurais,lags=40)
plt.show()

### Redes neurais Pytorch

In [ ]:
with torch.no_grad():
    modelo.eval()
    predi_pytorch_treino = modelo(torch.Tensor(x_treino)).numpy().flatten().tolist()

In [ ]:
residuo_pytorch = y_treino - predi_pytorch_treino

In [ ]:
plt.hist(residuo_pytorch,bins=50);

In [ ]:
sm.graphics.tsa.plot_acf(residuo_pytorch,lags=40)
plt.show()


##Avaliando os dados de treino

In [ ]:
metrics_random_treino = evaluate_regression(y_treino,predic_rando_treino, 'Random Forest Regression')
metrics_linear_treino = evaluate_regression(y_treino,predic_treino_lasso, 'Linear Regression')
metrics_grad_treino = evaluate_regression(y_treino,predict_grad_treino, 'Gradiente Boosting Regression')
metrics_redes_treino = evaluate_regression(y_treino,predict_redes_treino, 'Redes Neurais')
metrics_torch_treino = evaluate_regression(y_treino,predi_pytorch_treino,'Pytorch')
metrics_treino = pd.concat([metrics_random_treino, metrics_linear_treino,metrics_grad_treino,metrics_redes_treino,metrics_torch_treino])
metrics_treino

In [ ]:
#Plotando as predicoes
plt.plot(treino.index,y_treino,label='Dados Reais')
plt.plot(treino.index,predict_redes_treino,label='Predição Redes Neurais')
plt.plot(treino.index,predic_treino_lasso,label='Regressão Lasso')
plt.plot(treino.index,predic_rando_treino,label='Predição Random Forest')
plt.plot(treino.index,predict_grad_treino,label='Gradient Boosting')
plt.plot(treino.index,predi_pytorch_treino,label='Pytorch')
plt.legend();

##Avaliando nos dados de teste

In [ ]:
predic_teste_lasso = lasso.predict(x_teste)

In [ ]:
predic_rando_teste = rfr.predict(x_teste)

In [ ]:
predict_grad_teste = grad_bost.predict(x_teste)

In [ ]:
predict_redes_teste = model.predict(x_teste)
predict_redes_teste = np.squeeze(predict_redes_teste)

In [ ]:
with torch.no_grad():
    modelo.eval()
    predi_pytorch_teste = modelo(torch.Tensor(x_teste)).numpy().flatten().tolist()

In [ ]:
metrics_random_teste = evaluate_regression(y_teste,predic_rando_teste, 'Random Forest Regression')
metrics_linear_teste = evaluate_regression(y_teste,predic_teste_lasso, 'Linear Regression')
metrics_grad_teste = evaluate_regression(y_teste,predict_grad_teste, 'Gradiente Boosting Regression')
metrics_redes_teste = evaluate_regression(y_teste,predict_redes_teste, 'Redes Neurais')
metrics_pytorch_teste = evaluate_regression(y_teste,predi_pytorch_teste, 'Pytorch')
metrics_teste = pd.concat([metrics_random_teste, metrics_linear_teste,metrics_grad_teste,metrics_redes_teste,metrics_pytorch_teste])
metrics_teste

In [ ]:
#Plotando as predicoes
plt.plot(teste.index,y_teste,label='Dados Reais')
plt.plot(teste.index,predict_redes_teste,label='Predição Redes Neurais')
plt.plot(teste.index,predic_teste_lasso,label='Regressão Lasso')
plt.plot(teste.index,predic_rando_teste,label='Predição Random Forest')
plt.plot(teste.index,predict_grad_teste,label='Gradient Boosting')
plt.plot(teste.index,predi_pytorch_teste,label='Pytorch')
plt.legend();

In [ ]:
import plotly.graph_objs as go

# Crie um objeto de figura
fig = go.Figure()

fig.update_layout(
    height=1000, # altura em pixels
    width=2000, # largura em pixels
)

# Adicione a série temporal real ao gráfico
fig.add_trace(go.Scatter(x=teste.index, y=y_teste, name='Dados Reais'))

# Adicione as previsões de cada modelo ao gráfico
fig.add_trace(go.Scatter(x=teste.index, y=predict_redes_teste, name='Predição Redes Neurais'))
fig.add_trace(go.Scatter(x=teste.index, y=predic_teste_lasso, name='Regressão Lasso'))
fig.add_trace(go.Scatter(x=teste.index, y=predic_rando_teste, name='Predição Random Forest'))
fig.add_trace(go.Scatter(x=teste.index, y=predict_grad_teste, name='Gradient Boosting'))
fig.add_trace(go.Scatter(x=teste.index, y=predi_pytorch_teste, name='Pytorch'))

# Adicione um título e etiquetas aos eixos
fig.update_layout(title='Previsões de diferentes modelos',
                   xaxis_title='Data',
                   yaxis_title='Valor')

# Mostre o gráfico
fig.show()


##Intervalo de previsao

In [ ]:
def intervalo_predicao_bootstraps(x_treino,y_treino,modelo_treinado,x_teste,index,predicao,nome_algoritimo,percentiles = [2.5, 97.5],n_bootstraps=1000):
  import numpy as np
  from sklearn.utils import resample

  # Bootstrap
  n_samples = len(x_treino)
  bootstrapped_predictions = np.zeros((n_bootstraps, len(x_teste)))
  
  if nome_algoritimo == 'Redes Neurais':
    for i in range(n_bootstraps):
      bootstrapped_data = resample(np.column_stack((x_treino, y_treino)), replace=True, n_samples=n_samples, random_state=i)
      bootstrapped_x_train, bootstrapped_y_train = bootstrapped_data[:, :-1], bootstrapped_data[:, -1]
      bootstrapped_model = modelo_treinado.fit(bootstrapped_x_train, bootstrapped_y_train, epochs=100,batch_size=100,verbose=0)
      bootstrapped_predictions[i] = np.squeeze(model.predict(x_teste))
  else:
    for i in range(n_bootstraps):
      bootstrapped_data = resample(np.column_stack((x_treino, y_treino)), replace=True, n_samples=n_samples, random_state=i)
      bootstrapped_x_train, bootstrapped_y_train = bootstrapped_data[:, :-1], bootstrapped_data[:, -1]
      bootstrapped_model = modelo_treinado.fit(bootstrapped_x_train, bootstrapped_y_train)
      bootstrapped_predictions[i] = bootstrapped_model.predict(x_teste)

  # Calcula intervalos de previsão
  lower = np.percentile(bootstrapped_predictions, percentiles[0], axis=0)
  upper = np.percentile(bootstrapped_predictions, percentiles[1], axis=0)

  intervalos = pd.DataFrame(index=index)
  intervalos['Predicao'] = predicao
  intervalos[f'lower({percentiles[0]})'] = lower
  intervalos[f'upper({percentiles[1]})'] = upper

  import matplotlib.pyplot as plt

  plt.plot(index,predicao,'*')
  plt.fill_between(index,lower, upper, color='gray', alpha=0.5, label=f'Intervalo de confiança de {percentiles[1]-percentiles[0]}%')
  plt.legend()
  plt.xlabel('Datas')
  plt.ylabel('Predição')
  plt.title('Previsões com intervalo de confiança para a'+' '+nome_algoritimo)
  plt.show()
  return intervalos

In [ ]:
intervalo_random = intervalo_predicao_bootstraps(x_treino,y_treino,rfr,x_teste,teste.index,predic_rando_teste,'Random_Forest')

In [ ]:
intervalo_random

In [ ]:
intervalo_lasso = intervalo_predicao_bootstraps(x_treino,y_treino,lasso,x_teste,teste.index,predic_teste_lasso,'Regressão Lasso')

In [ ]:
intervalo_lasso

In [ ]:
intervalo_grad = intervalo_predicao_bootstraps(x_treino,y_treino,grad_bost,x_teste,teste.index,predict_grad_teste,'Gradiente Boosting',n_bootstraps=100)

In [ ]:
intervalo_grad

In [ ]:
intervalo_redes = intervalo_predicao_bootstraps(x_treino,y_treino,model,x_teste,teste.index,predict_redes_teste,'Redes Neurais',n_bootstraps=20)

In [ ]:
intervalo_redes

##Predicao valores

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# gerar a faixa de valores de predição
X_pred = np.linspace(0, 10, 100).reshape(-1, 1) # aqui estamos gerando 100 pontos igualmente espaçados entre 0 e 10

# fazer as previsões para a faixa de valores
predicao_lasso = lasso.predict(dados_predicao.values)

# calcular o desvio padrão dos resíduos
desvio_padrao_residuos = np.std(residuo_lasso)

# calcular o intervalo de previsão para a faixa de valores
nivel_confianca = 0.95
z = 1.96 # valor crítico para um nível de confiança de 95%
intervalo_previsao = z * desvio_padrao_residuos * np.sqrt(1 + np.diag(np.dot(dados_predicao.values, np.dot(np.linalg.inv(np.dot(x_treino.T, x_treino)), dados_predicao.values.T))))

index = dados_predicao.index.strftime('%Y-%m')
intervalos = pd.DataFrame()
intervalos['predicao'] = predicao_lasso
intervalos['nivel_baixo'] = (predicao_lasso - intervalo_previsao)
intervalos['nivel_alto'] = (predicao_lasso + intervalo_previsao) 

index = dados_predicao.index.strftime('%Y-%m')
# plotar a faixa de valores e o intervalo de previsão correspondente
plt.plot(index, predicao_lasso, label='Previsão')
plt.fill_between(index,predicao_lasso - intervalo_previsao, predicao_lasso + intervalo_previsao, alpha=0.2, label='Intervalo de previsão')
plt.legend()
plt.xlabel('Variável independente')
plt.ylabel('Variável dependente')
plt.title('Intervalo de previsão para valores futuros')
plt.show()


In [ ]:
def intervalo_predicao(residuo,dados_predicao,x_treino,predicao,nivel_confianca):
  import numpy as np
  import matplotlib.pyplot as plt

  # gerar a faixa de valores de predição
  X_pred = np.linspace(0, 10, 100).reshape(-1, 1) # aqui estamos gerando 100 pontos igualmente espaçados entre 0 e 10

  # calcular o desvio padrão dos resíduos
  desvio_padrao_residuos = np.std(residuo)

  # calcular o intervalo de previsão para a faixa de valores
  # calcular o intervalo de previsão para a faixa de valores
  z = 0.0
  if nivel_confianca == 0.95:
    z = 1.96 # valor crítico para um nível de confiança de 95%
  elif nivel_confianca == 0.90:
    z = 1.645 # valor crítico para um nível de confiança de 90%
  else:
    z = 1.0 # valor crítico para um nível de confiança de 68%

  intervalo_previsao = z * desvio_padrao_residuos * np.sqrt(1 + np.diag(np.dot(dados_predicao.values, np.dot(np.linalg.inv(np.dot(x_treino.T, x_treino)), dados_predicao.values.T))))
  
  index = dados_predicao.index.strftime('%Y-%m')
  intervalos = pd.DataFrame()
  intervalos['predicao'] = predicao
  intervalos[f'nivel_baixo({nivel_confianca})'] = (predicao - intervalo_previsao)
  intervalos[f'nivel_alto({nivel_confianca})'] = (predicao + intervalo_previsao) 
    # plotar a faixa de valores e o intervalo de previsão correspondente
  plt.plot(index, predicao, label='Previsão')
  plt.fill_between(index,predicao - intervalo_previsao, predicao + intervalo_previsao, alpha=0.2, label='Intervalo de previsão')
  plt.legend()
  plt.xlabel('Variável independente')
  plt.ylabel('Variável dependente')
  plt.title(f'Intervalo de previsão para valores futuros de {nivel_confianca}')
  plt.show()

  return intervalos

In [ ]:
predicao_lasso = lasso.predict(dados_predicao)
intervalos_lasso = intervalo_predicao(residuo_lasso,dados_predicao,x_treino,predicao_lasso,0.95)
intervalos_lasso

In [ ]:
predicao_random = rfr.predict(dados_predicao)
intervalos_random = intervalo_predicao(residuo_random,dados_predicao,x_treino,predicao_random,0.95)
intervalos_random

In [ ]:
predicao_boost = grad_bost.predict(dados_predicao)
intervalos_boost= intervalo_predicao(residuo_grad,dados_predicao,x_treino,predicao_boost,0.95)
intervalos_boost

In [ ]:
predicao_redes = np.squeeze(model.predict(dados_predicao))
intervalos_redes= intervalo_predicao(residuo_neurais,dados_predicao,x_treino,predicao_redes,0.95)
intervalos_redes

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_